# The TEGR1 Data Set


This is a streamlined data ingress for 1 round of QF. This is the expected dataset from an organization running QF like GitCoin or Giveth. 

This data is being used to implement a digital twin environment for analysis with the Token Engineering Commons. 

In [1]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
import holoviews as hv
import param as pm

## Loading The Data

In [45]:
df = pd.read_csv('output/TEGR1.csv')
df.head()

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,tec_tokens_flag,balance_tea,tea_flag
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,28.57,0.0,0.0,3.0,1.0
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,27.21,0.0,0.0,0.0,0.0
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,28.57,0.0,0.0,0.0,0.0
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0


Number of Donations, Number of Projects, and Number of Unique donors. All as integers. 

In [46]:
# Inspect number of unique addresses
df.select_dtypes(include=['object']).nunique()

id              257
projectId        16
voter            86
grantAddress     16
dtype: int64

Overview of Quadradic Funding Data

In [47]:
df[['amountUSD', 'coefficient', 'rawScore']].describe()

,amountUSD,coefficient,rawScore
count,257.000000,257.000000,257.000000
mean,23.899090,1.235409,32.010817
std,82.941205,0.250061,12.050044
min,0.368890,1.000000,3.230000
25%,3.000196,1.000000,22.970000
50%,5.998856,1.000000,28.570000
75%,12.346883,1.500000,40.480000
max,936.545861,1.500000,55.060000


Overview of the TEC SME data.

In [48]:
df[['balance_tec', 'tec_tokens_flag', 'balance_tea', 'tea_flag']].describe()

,balance_tec,tec_tokens_flag,balance_tea,tea_flag
count,257.000000,257.000000,257.000000,257.000000
mean,5420.926507,0.420233,0.268482,0.171206
std,14848.071173,0.494559,1.086988,0.377424
min,0.000000,0.000000,-1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,1210.915986,1.000000,0.000000,0.000000
max,73838.661487,1.000000,5.000000,1.000000


## Explore the Vote Coefficients Input Dataset Using Hvplot Explorer

In [5]:
hvexplorer = hvplot.explorer(
    df_qf, 
    height=400)
hvexplorer.param.set_param(kind='step', x='index', y_multi=['amountUSD', 'rawScore'], by=[])
hvexplorer.labels.title = 'TEC Quadratic Funding Round #1 Data'
hvexplorer.labels.xlabel = 'Index'
hvexplorer.labels.ylabel = 'USD Amount and Raw Score'
hvexplorer

hvDataFrameExplorer(axes=Axes(explorer=..., height=400, legend='right', logx=False, logy=False, name='Axes00829', responsive=False, shared_axes=True, width=None, xlim=None, ylim=None), by=[], colormapping=Colormapping(clim=None, cmap='kbc_r', cnorm='linear', color=None, colorbar=None, explorer=..., name='Colormapping00889', rescale_discrete_levels=True, symmetric=False), groupby=[], kind='step', labels=Labels(clabel='', explorer=..., fontscale=1, name='Labels00936', rot=0, title='TEC Quadratic Funding Round #1 Data', xlabel='Index', ylabel='USD Amount and Raw Score'), name='hvDataFrameExplorer00796', operations=Operations(aggregator=None, datashade=False, dynspread=False, explorer=..., name='Operations00980', rasterize=False, x_sampling=None, y_sampling=None), style=Style(alpha=1, explorer=..., name='Style01024'), x='index', y='id', y_multi=['amountUSD', 'rawScore'], z=None)

In [6]:
# hvexplorer.param.set_param(kind='scatter', x='rawScore', y_multi=['amountUSD'], by=['projectId'])
# hvexplorer.labels.xlabel = 'Raw Score'
# hvexplorer.labels.ylabel = 'Amount USD'

## Applying the QF Algorithm
Inspired by Octopus at https://forum.tecommons.org/t/strengths-and-weaknesses-of-conviction-voting-and-other-mechanisms/1278

```python
def quadratic_fund(str: project) -> float:
    allocation = np.square(np.sum([sqrt(agent.votes.get(project)) for agent in system.agents]))
    return allocation
```

In [49]:
df['sqrt(amountUSD)'] = np.sqrt(df['amountUSD'])
df['sum(sqrt(amountUSD))'] = df.groupby('projectId')['sqrt(amountUSD)'].transform('sum')
df['sq(sum(sqrt(amountUSD)))'] = df['sum(sqrt(amountUSD))'].transform(lambda x: x**2)
df['quadradic_allocation'] = df['sq(sum(sqrt(amountUSD)))'] / df['sq(sum(sqrt(amountUSD)))'].sum()
df['default_allocation'] = df['amountUSD'] / df['amountUSD'].sum()
df['quadratic_amount'] = df['amountUSD'].sum() * df['quadradic_allocation']

In [50]:
df

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,tec_tokens_flag,balance_tea,tea_flag,sqrt(amountUSD),sum(sqrt(amountUSD)),sq(sum(sqrt(amountUSD))),quadradic_allocation,default_allocation,quadratic_amount
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,28.57,0.0,0.0,3.0,1.0,3.030566,64.723106,4189.080414,0.003203,0.001495,19.673692
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,27.21,0.0,0.0,0.0,0.0,2.023504,15.075191,227.261396,0.000174,0.000667,1.067316
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,28.57,0.0,0.0,0.0,0.0,8.628248,56.397388,3180.665361,0.002432,0.012121,14.937749
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0,2.362943,47.809680,2285.765459,0.001748,0.000909,10.734921
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0,2.362943,62.051660,3850.408505,0.002944,0.000909,18.083146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,0x26e1e300,0x97589cd1,7,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,29.74,0.0,0.0,0.0,0.0,1.359339,56.397388,3180.665361,0.002432,0.000301,14.937749
253,0xa21ca1aa,0xec026845,16,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,21.07,0.0,0.0,0.0,0.0,1.357863,47.809680,2285.765459,0.001748,0.000300,10.734921
254,0x634b5156,0xf1f4942d,24,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,21.07,0.0,0.0,0.0,0.0,1.357863,46.200650,2134.500081,0.001632,0.000300,10.024514
255,0x4efa29aa,0xcf3165f4,10,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,18.04,0.0,0.0,0.0,0.0,1.357863,7.801686,60.866307,0.000047,0.000300,0.285854


## Automating the algorithm.

In [8]:
df = pd.read_csv('output/TEGR1.csv')
df

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,tec_tokens_flag,balance_tea,tea_flag
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,28.57,0.0,0.0,3.0,1.0
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,27.21,0.0,0.0,0.0,0.0
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,28.57,0.0,0.0,0.0,0.0
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,0x26e1e300,0x97589cd1,7,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,29.74,0.0,0.0,0.0,0.0
253,0xa21ca1aa,0xec026845,16,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,21.07,0.0,0.0,0.0,0.0
254,0x634b5156,0xf1f4942d,24,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,21.07,0.0,0.0,0.0,0.0
255,0x4efa29aa,0xcf3165f4,10,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,18.04,0.0,0.0,0.0,0.0


### The QF transformer.

In [53]:
def qf(df, column_name='amountUSD', new_column_name='quadratic_amount'):
    df = df.copy(deep=True)
    df[f'{column_name}_allocation'] = df[column_name] / df[column_name].sum()
    df[f'sqrt({column_name})'] = np.sqrt(df[column_name])
    df[f'sum(sqrt({column_name}))'] = df.groupby('projectId')[f'sqrt({column_name})'].transform('sum')
    df[f'sq(sum(sqrt({column_name})))'] = df[f'sum(sqrt({column_name}))'].transform(lambda x: x**2)
    df[f'{new_column_name}_allocation'] = df[f'sq(sum(sqrt({column_name})))'] / df[f'sq(sum(sqrt({column_name})))'].sum()
    df[new_column_name] = df[column_name].sum() * df[f'{new_column_name}_allocation']
    
    return df

In [54]:
df = qf(df)
df

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,...,balance_tea,tea_flag,sqrt(amountUSD),sum(sqrt(amountUSD)),sq(sum(sqrt(amountUSD))),quadradic_allocation,default_allocation,quadratic_amount,amountUSD_allocation,quadratic_amount_allocation
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,28.57,0.0,...,3.0,1.0,3.030566,64.723106,4189.080414,0.003203,0.001495,19.673692,0.001495,0.003203
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,27.21,0.0,...,0.0,0.0,2.023504,15.075191,227.261396,0.000174,0.000667,1.067316,0.000667,0.000174
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,28.57,0.0,...,0.0,0.0,8.628248,56.397388,3180.665361,0.002432,0.012121,14.937749,0.012121,0.002432
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,23.56,0.0,...,0.0,0.0,2.362943,47.809680,2285.765459,0.001748,0.000909,10.734921,0.000909,0.001748
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,23.56,0.0,...,0.0,0.0,2.362943,62.051660,3850.408505,0.002944,0.000909,18.083146,0.000909,0.002944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,0x26e1e300,0x97589cd1,7,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,29.74,0.0,...,0.0,0.0,1.359339,56.397388,3180.665361,0.002432,0.000301,14.937749,0.000301,0.002432
253,0xa21ca1aa,0xec026845,16,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,21.07,0.0,...,0.0,0.0,1.357863,47.809680,2285.765459,0.001748,0.000300,10.734921,0.000300,0.001748
254,0x634b5156,0xf1f4942d,24,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,21.07,0.0,...,0.0,0.0,1.357863,46.200650,2134.500081,0.001632,0.000300,10.024514,0.000300,0.001632
255,0x4efa29aa,0xcf3165f4,10,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,18.04,0.0,...,0.0,0.0,1.357863,7.801686,60.866307,0.000047,0.000300,0.285854,0.000300,0.000047


### SME Sginal Boosting

In [55]:
# Compute the boosted allocation
df['amount_boosted'] = df['amountUSD'] * df['coefficient']
df = qf(df, column_name='amount_boosted', new_column_name='quadratic_amount_boosted')
df

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,...,quadratic_amount,amountUSD_allocation,quadratic_amount_allocation,amount_boosted,amount_boosted_allocation,sqrt(amount_boosted),sum(sqrt(amount_boosted)),sq(sum(sqrt(amount_boosted))),quadratic_amount_boosted_allocation,quadratic_amount_boosted
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,28.57,0.0,...,19.673692,0.001495,0.003203,13.776499,0.001779,3.711671,74.912721,5611.915710,0.003498,27.080157
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,27.21,0.0,...,1.067316,0.000667,0.000174,4.094567,0.000529,2.023504,16.321380,266.387447,0.000166,1.285446
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,28.57,0.0,...,14.937749,0.012121,0.002432,74.446665,0.009616,8.628248,63.587712,4043.397162,0.002520,19.511310
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,23.56,0.0,...,10.734921,0.000909,0.001748,5.583500,0.000721,2.362943,53.638466,2877.085023,0.001793,13.883301
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,23.56,0.0,...,18.083146,0.000909,0.002944,5.583500,0.000721,2.362943,70.476216,4966.897020,0.003096,23.967635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,0x26e1e300,0x97589cd1,7,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,29.74,0.0,...,14.937749,0.000301,0.002432,1.847803,0.000239,1.359339,63.587712,4043.397162,0.002520,19.511310
253,0xa21ca1aa,0xec026845,16,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,21.07,0.0,...,10.734921,0.000300,0.001748,1.843793,0.000238,1.357863,53.638466,2877.085023,0.001793,13.883301
254,0x634b5156,0xf1f4942d,24,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,21.07,0.0,...,10.024514,0.000300,0.001632,1.843793,0.000238,1.357863,48.695289,2371.231134,0.001478,11.442316
255,0x4efa29aa,0xcf3165f4,10,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,18.04,0.0,...,0.285854,0.000300,0.000047,1.843793,0.000238,1.357863,8.337738,69.517878,0.000043,0.335457


### The Allocations Dataset
Sum qf allocations by project.

In [57]:
# Examine the project allocations
allocations = df[['projectId'] + list(df.columns[df.columns.str.contains('allocation')])].groupby('projectId').sum().drop('amount_boosted_allocation',axis=1)
allocations

,quadradic_allocation,default_allocation,amountUSD_allocation,quadratic_amount_allocation,quadratic_amount_boosted_allocation
projectId,,,,,
0x10b3f00e,0.001472,0.007078,0.007078,0.001472,0.001487
0x23387567,0.035079,0.163461,0.163461,0.035079,0.040895
0x4cd41869,0.189890,0.085993,0.085993,0.189890,0.158764
0x5351510d,0.002201,0.007997,0.007997,0.002201,0.002275
0x64a30a4b,0.057656,0.062008,0.062008,0.057656,0.062962
0x72b0d6a6,0.004467,0.011389,0.011389,0.004467,0.005040
0x8d6f0c7b,0.078177,0.037500,0.037500,0.078177,0.085536
0x97589cd1,0.046209,0.044694,0.044694,0.046209,0.047885
0xa9bdf738,0.073604,0.031311,0.031311,0.073604,0.077397


Verify that all allocations sum to one.

In [58]:
allocations.sum()

quadradic_allocation                   1.0
default_allocation                     1.0
amountUSD_allocation                   1.0
quadratic_amount_allocation            1.0
quadratic_amount_boosted_allocation    1.0
dtype: float64

### The Results Dataset

In [61]:
donations = df['amountUSD'].sum()
df_donations = allocations * donations

matching_pool = 25_000
funding_pool = matching_pool + donations
df_funding_pool = allocations * funding_pool

results = df_donations.merge(df_funding_pool, left_index=True, right_index=True, suffixes=('_unmatched', '_matched'))
results = results.sort_values('quadratic_amount_allocation_unmatched', ascending=False)

In [62]:
results

,quadradic_allocation_unmatched,default_allocation_unmatched,amountUSD_allocation_unmatched,quadratic_amount_allocation_unmatched,quadratic_amount_boosted_allocation_unmatched,quadradic_allocation_matched,default_allocation_matched,amountUSD_allocation_matched,quadratic_amount_allocation_matched,quadratic_amount_boosted_allocation_matched
projectId,,,,,,,,,,
0xe6424ab2,1309.615374,611.890594,611.890594,1309.615374,1402.811031,6640.131798,3102.463724,3102.463724,6640.131798,7112.660951
0x4cd41869,1166.316368,528.174328,528.174328,1166.316368,975.136198,5913.564056,2677.997845,2677.997845,5913.564056,4944.224851
0xdd9b885d,1106.412911,2041.312405,2041.312405,1106.412911,1028.524764,5609.836063,10350.049091,10350.049091,5609.836063,5214.920444
0x8d6f0c7b,480.165901,230.329281,230.329281,480.165901,525.364722,2434.581121,1167.836611,1167.836611,2434.581121,2663.752324
0xa9bdf738,452.078638,192.313869,192.313869,452.078638,475.374453,2292.170509,975.087387,975.087387,2292.170509,2410.287086
0xe8249a10,392.339044,263.524009,263.524009,392.339044,425.977411,1989.273351,1336.143561,1336.143561,1989.273351,2159.829678
0x64a30a4b,354.126454,380.857253,380.857253,354.126454,386.717940,1795.524379,1931.057322,1931.057322,1795.524379,1960.772711
0x97589cd1,283.817225,274.514986,274.514986,283.817225,294.110600,1439.036084,1391.871022,1391.871022,1439.036084,1491.226499
0x23387567,215.460068,1003.988620,1003.988620,215.460068,251.178482,1092.445365,5090.515042,5090.515042,1092.445365,1273.548143


## Visualizing the Results

Replicating Bar Chart #1 from the Medium Article.

In [63]:
results.rename({
    'quadratic_amount_allocation_unmatched': 'QF Unmatched', 
    'quadratic_amount_allocation_matched': 'QF Matched', 
    'quadratic_amount_boosted_allocation_matched': 'QF Matched + SME',
}, axis=1).hvplot.bar(
    y=['QF Unmatched', 'QF Matched', 'QF Matched + SME'],
    rot=45,
    stacked=False,
    title="Adding Expertise into the QF Signal",
).opts(multi_level=False)

:Bars   [projectId,Variable]   (value)

Replicating Bar Chart #2 from the Medium Article.

In [64]:
results['Percentage Boost'] = 100 * ((results['quadratic_amount_boosted_allocation_matched'] - results['quadratic_amount_allocation_matched']) / results['quadratic_amount_allocation_matched'] + 1)

In [65]:

results.hvplot.bar(
    y='Percentage Boost', 
    color='purple', 
    ylim=(0, 180), 
    yformatter="%.0f%%", 
    yticks=list(range(0,200,20)),
    grid=True,
    height=400,
    rot=45,
    title="SME Boost as % of QF Boost by Project",
) * hv.HLine(100)

:Overlay
   .Bars.I  :Bars   [projectId]   (Percentage Boost)
   .HLine.I :HLine   [x,y]

In [66]:
results

,quadradic_allocation_unmatched,default_allocation_unmatched,amountUSD_allocation_unmatched,quadratic_amount_allocation_unmatched,quadratic_amount_boosted_allocation_unmatched,quadradic_allocation_matched,default_allocation_matched,amountUSD_allocation_matched,quadratic_amount_allocation_matched,quadratic_amount_boosted_allocation_matched,Percentage Boost
projectId,,,,,,,,,,,
0xe6424ab2,1309.615374,611.890594,611.890594,1309.615374,1402.811031,6640.131798,3102.463724,3102.463724,6640.131798,7112.660951,107.116262
0x4cd41869,1166.316368,528.174328,528.174328,1166.316368,975.136198,5913.564056,2677.997845,2677.997845,5913.564056,4944.224851,83.608207
0xdd9b885d,1106.412911,2041.312405,2041.312405,1106.412911,1028.524764,5609.836063,10350.049091,10350.049091,5609.836063,5214.920444,92.960300
0x8d6f0c7b,480.165901,230.329281,230.329281,480.165901,525.364722,2434.581121,1167.836611,1167.836611,2434.581121,2663.752324,109.413168
0xa9bdf738,452.078638,192.313869,192.313869,452.078638,475.374453,2292.170509,975.087387,975.087387,2292.170509,2410.287086,105.153045
0xe8249a10,392.339044,263.524009,263.524009,392.339044,425.977411,1989.273351,1336.143561,1336.143561,1989.273351,2159.829678,108.573800
0x64a30a4b,354.126454,380.857253,380.857253,354.126454,386.717940,1795.524379,1931.057322,1931.057322,1795.524379,1960.772711,109.203347
0x97589cd1,283.817225,274.514986,274.514986,283.817225,294.110600,1439.036084,1391.871022,1391.871022,1439.036084,1491.226499,103.626762
0x23387567,215.460068,1003.988620,1003.988620,215.460068,251.178482,1092.445365,5090.515042,5090.515042,1092.445365,1273.548143,116.577742


# Parameterization and App Development

In [72]:
class TECQFSME(pm.Parameterized):
    boosting = pm.Number(1, bounds=(0,2), step=0.1)
    dataset  = pm.DataFrame(precedence=-1)
    matching_pool = pm.Integer(25_000, bounds=(0, 100_000), step=5_000)
    donations = pm.Number(0, constant=True)
    funding_pool = pm.Number(0, constant=True)
    allocations  = pm.DataFrame(precedence=-1)
    results  = pm.DataFrame(precedence=-1)
    
    def __init__(self, **params):
        super().__init__(**params)  
        self.update()
        
    @staticmethod
    def qf(df, column_name='amountUSD', new_column_name='quadratic_amount'):
        df = df.copy(deep=True)
        df[f'{column_name}_allocation'] = df[column_name] / df[column_name].sum()
        df[f'sqrt({column_name})'] = np.sqrt(df[column_name])
        df[f'sum(sqrt({column_name}))'] = df.groupby('projectId')[f'sqrt({column_name})'].transform('sum')
        df[f'sq(sum(sqrt({column_name})))'] = df[f'sum(sqrt({column_name}))'].transform(lambda x: x**2)
        df[f'{new_column_name}_allocation'] = df[f'sq(sum(sqrt({column_name})))'] / df[f'sq(sum(sqrt({column_name})))'].sum()
        df[new_column_name] = df[column_name].sum() * df[f'{new_column_name}_allocation']
        return df
    
    @pm.depends('boosting', 'matching_pool', watch=True)
    def update(self):
        # Update total donations and funding pool
        with pm.edit_constant(self):
            self.donations = self.dataset['amountUSD'].sum()
            self.funding_pool = self.matching_pool + self.donations
        
        with pm.parameterized.batch_call_watchers(self):
            # Update the Coefficient
            self.dataset['coefficient'] = 1 + self.boosting * (self.dataset['tec_tokens_flag'].astype(int) | self.dataset['tea_flag'].astype(int))

            # Apply the Coefficient
            self.dataset['amount_boosted'] = self.dataset['amountUSD'] * self.dataset['coefficient']

            # Compute the Boosted Allocation
            self.dataset = self.qf(self.dataset, column_name='amount_boosted', new_column_name='quadratic_amount_boosted')

            # Remove the intermediate steps
            self.dataset = self.dataset[self.dataset.columns[~self.dataset.columns.str.contains('sqrt')]]

            # Examine the project allocations
            self.allocations = self.dataset[['projectId'] + list(self.dataset.columns[self.dataset.columns.str.contains('allocation')])].groupby('projectId').sum().drop('amount_boosted_allocation',axis=1)

            # Save the results sort by quadratic funding amounts
            self.results = (self.donations * self.allocations).merge((self.funding_pool * self.allocations), left_index=True, right_index=True, suffixes=('_unmatched', '_matched'))
            self.results = self.results.sort_values('quadratic_amount_allocation_unmatched', ascending=False)

            # Save the boosting percentage stat
            self.results['Percentage Boost'] = 100 * ((self.results['quadratic_amount_boosted_allocation_matched'] - self.results['quadratic_amount_allocation_matched']) / self.results['quadratic_amount_allocation_matched'] + 1)

        
    @pm.depends('dataset')
    def view_expertise_signal(self):
        return self.results.rename({
            'quadratic_amount_allocation_unmatched': 'QF Unmatched', 
            'quadratic_amount_allocation_matched': 'QF Matched', 
            'quadratic_amount_boosted_allocation_matched': 'QF Matched + SME',
        }, axis=1).hvplot.bar(
            y=['QF Unmatched', 'QF Matched', 'QF Matched + SME'],
            rot=45,
            stacked=False,
            title="Adding Expertise into the QF Signal",
        ).opts(multi_level=False, legend_position='top_right')
    
    @pm.depends('dataset')
    def view_percentage_boost(self):
        return self.results.hvplot.bar(
            y='Percentage Boost', 
            color='purple', 
            ylim=(0, 180), 
            yformatter="%.0f%%", 
            yticks=list(range(0,200,20)),
            grid=True,
            height=400,
            rot=45,
            title="SME Boost as % of QF Boost by Project",
        ) * hv.HLine(100)
        
    
    def view(self):
        return pn.Row(self.param, self.view_percentage_boost)

In [73]:
df

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,...,quadratic_amount,amountUSD_allocation,quadratic_amount_allocation,amount_boosted,amount_boosted_allocation,sqrt(amount_boosted),sum(sqrt(amount_boosted)),sq(sum(sqrt(amount_boosted))),quadratic_amount_boosted_allocation,quadratic_amount_boosted
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,28.57,0.0,...,19.673692,0.001495,0.003203,13.776499,0.001779,3.711671,74.912721,5611.915710,0.003498,27.080157
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,27.21,0.0,...,1.067316,0.000667,0.000174,4.094567,0.000529,2.023504,16.321380,266.387447,0.000166,1.285446
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,28.57,0.0,...,14.937749,0.012121,0.002432,74.446665,0.009616,8.628248,63.587712,4043.397162,0.002520,19.511310
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,23.56,0.0,...,10.734921,0.000909,0.001748,5.583500,0.000721,2.362943,53.638466,2877.085023,0.001793,13.883301
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,23.56,0.0,...,18.083146,0.000909,0.002944,5.583500,0.000721,2.362943,70.476216,4966.897020,0.003096,23.967635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,0x26e1e300,0x97589cd1,7,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,29.74,0.0,...,14.937749,0.000301,0.002432,1.847803,0.000239,1.359339,63.587712,4043.397162,0.002520,19.511310
253,0xa21ca1aa,0xec026845,16,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,21.07,0.0,...,10.734921,0.000300,0.001748,1.843793,0.000238,1.357863,53.638466,2877.085023,0.001793,13.883301
254,0x634b5156,0xf1f4942d,24,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,21.07,0.0,...,10.024514,0.000300,0.001632,1.843793,0.000238,1.357863,48.695289,2371.231134,0.001478,11.442316
255,0x4efa29aa,0xcf3165f4,10,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,18.04,0.0,...,0.285854,0.000300,0.000047,1.843793,0.000238,1.357863,8.337738,69.517878,0.000043,0.335457


In [74]:
tec_qf_sme = TECQFSME(dataset=df.copy(deep=True))

In [75]:
tec_qf_sme.view()

Row
    [0] Column(margin=(5, 10), name='TECQFSME')
        [0] StaticText(value='<b>TECQFSME</b>')
        [1] FloatSlider(end=2, name='Boosting', value=1)
        [2] IntSlider(end=100000, name='Matching pool', step=5000, value=25000)
        [3] FloatInput(disabled=True, name='Donations', value=6142.066124660001)
        [4] FloatInput(disabled=True, name='Funding pool', value=31142.06612466)
    [1] ParamMethod(method, _pane=HoloViews, defer_load=False)

In [76]:
tec_qf_sme.view_expertise_signal()

:Bars   [projectId,Variable]   (value)